# Practicumopdrachten Week 2.3 (eerste kans)

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige practicum de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [ ]:
import pandas as pd
import sqlite3 as s3
import warnings

Zet het bestand go_sales.sqlite in hetzelfde mapje als dit notebook.

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige practicum de code om een connectie met het bestand go_sales.sqlite te maken.

In [ ]:
go_sales = s3.connect('go_sales.sqlite')

Lees van de ingelezen go_sales-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- target
- returned_item
- return_reason

In [ ]:
product = pd.read_sql_query('SELECT * FROM product', go_sales)
product_type = pd.read_sql_query('SELECT * FROM product_type', go_sales)
product_line = pd.read_sql_query('SELECT * FROM product_line', go_sales)
sales_staff = pd.read_sql_query('SELECT * FROM sales_staff', go_sales)
sales_branch = pd.read_sql_query('SELECT * FROM sales_branch', go_sales)
retailer_site = pd.read_sql_query('SELECT * FROM retailer_site', go_sales)
country = pd.read_sql_query('SELECT * FROM country', go_sales)
order_header = pd.read_sql_query('SELECT * FROM order_header', go_sales)
order_details = pd.read_sql_query('SELECT * FROM order_details', go_sales)
sales_target = pd.read_sql_query('SELECT * FROM sales_targetData', go_sales)
returned_item = pd.read_sql_query('SELECT * FROM returned_item', go_sales)

Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [ ]:
cost_range = (product['PRODUCTION_COST'].astype(float) < 100) & (product['PRODUCTION_COST'].astype(float) > 50)
pd.DataFrame(product.loc[cost_range, ['PRODUCT_NAME', 'PRODUCTION_COST']])

Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [ ]:
margin_range = (product['MARGIN'].astype(float) < .20) | (product['MARGIN'].astype(float) > .60)
pd.DataFrame(product.loc[margin_range, ['PRODUCT_NAME', 'MARGIN']])

Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [ ]:
currency = country['CURRENCY_NAME'] == 'francs'
pd.DataFrame(country.loc[currency, ['COUNTRY']].sort_values(by=['COUNTRY']))

Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [ ]:
filter = product['MARGIN'].astype(float) > .50
pd.DataFrame(product.loc[filter, ['INTRODUCTION_DATE']].drop_duplicates())

Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 28 rijen)

In [ ]:
filter = (sales_branch['ADDRESS2'].notna()) & (sales_branch['REGION'].notna())
pd.DataFrame(sales_branch.loc[filter, ['ADDRESS1', 'CITY']])

Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

In [ ]:
currency = (country['CURRENCY_NAME'] == 'new dollar') | (country['CURRENCY_NAME'] == 'dollars')
pd.DataFrame(country.loc[currency, ['COUNTRY']].sort_values(by=['COUNTRY']))

Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

In [ ]:
filter = (retailer_site['POSTAL_ZONE'].str.startswith('D')) & (retailer_site['ADDRESS2'].notna())
pd.DataFrame(retailer_site.loc[filter, ['ADDRESS1', 'ADDRESS2', 'CITY']])

## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

In [ ]:
len(returned_item)

Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

In [ ]:
filter = retailer_site[retailer_site['REGION'].notna()]
filter['REGION'].drop_duplicates().count()

Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

In [ ]:
pd.DataFrame({ 'laagste' : [product['MARGIN'].astype(float).min()], 'hoogste' : [product['MARGIN'].astype(float).max()], 'gemiddelde' : [product['MARGIN'].astype(float).mean()] })

Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

In [ ]:
retailer_site['ADDRESS2'].isna().sum()

Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

In [ ]:
filter = order_details['UNIT_SALE_PRICE'].astype(float) < order_details['UNIT_PRICE'].astype(float)
filtered = pd.DataFrame(order_details.loc[filter, :])
filtered['UNIT_COST'].astype(float).mean()

Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

In [ ]:
filter = sales_staff['POSITION_EN'].value_counts().reset_index()
filter.columns = ['POSITION_EN', 'COUNT']
filter

Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

In [ ]:
phone_counts = sales_staff['WORK_PHONE'].value_counts()
filter = phone_counts > 4
result = phone_counts[filter].reset_index()
result.columns = ['WORK_PHONE', 'COUNT']
result

## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

In [ ]:
merged = pd.merge(retailer_site, country, left_on = 'COUNTRY_CODE', how = 'inner', right_on = 'COUNTRY_CODE')
filter = merged['COUNTRY'] == 'Netherlands'
pd.DataFrame(retailer_site.loc[filter, ['ADDRESS1', 'CITY']])

Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

In [ ]:
merged = pd.merge(product, product_type, left_on = 'PRODUCT_TYPE_CODE', how = 'inner', right_on = 'PRODUCT_TYPE_CODE')
filter = merged['PRODUCT_TYPE_EN'] == 'Eyewear'
pd.DataFrame(merged.loc[filter, ['PRODUCT_NAME']])

Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

In [ ]:
merged = pd.merge(sales_staff, sales_branch, left_on='SALES_BRANCH_CODE', how='inner', right_on='SALES_BRANCH_CODE')
filter = merged['POSITION_EN'] == 'Branch Manager'
pd.DataFrame(merged.loc[filter, ['ADDRESS1', 'FIRST_NAME', 'LAST_NAME']])

Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 9 rijen) 

In [ ]:
merged = pd.merge(sales_staff, order_header, left_on='SALES_STAFF_CODE', how='outer', right_on='SALES_STAFF_CODE')
filter = (merged['POSITION_EN'].str.contains('Manager')) #& (merged['ORDER_NUMBER'].notna())
pd.DataFrame(merged.loc[filter, ['POSITION_EN', 'ORDER_DATE']].drop_duplicates())

Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

In [ ]:
product_merged = pd.merge(product, product_type, left_on='PRODUCT_TYPE_CODE', how='inner', right_on='PRODUCT_TYPE_CODE')
merged = pd.merge(product_merged, order_details, left_on='PRODUCT_NUMBER', how='inner', right_on='PRODUCT_NUMBER')
filter = merged['QUANTITY'].astype(int) > 750
pd.DataFrame(merged.loc[filter, ['PRODUCT_NAME', 'PRODUCT_TYPE_EN']].drop_duplicates())

Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

In [ ]:
order_details

Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

In [ ]:
merged = pd.merge(product, product_type, on='PRODUCT_TYPE_CODE')
merged.groupby('PRODUCT_TYPE_EN').size().reset_index(name='PRODUCT_COUNT')

Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

In [ ]:
merged = pd.merge(retailer_site, country, on='COUNTRY_CODE')
merged.groupby('COUNTRY_CODE').size().reset_index(name='BRANCH_COUNT')

Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (4 kolommen, 10 rijen) 

In [ ]:
merged = pd.merge(product, product_type, on='PRODUCT_TYPE_CODE')
cooking_gear_data = merged[merged['PRODUCT_TYPE_EN'] == 'Cooking Gear']
order_details['UNIT_SALE_PRICE'] = order_details['UNIT_SALE_PRICE'].astype(float)
summary = order_details.groupby('PRODUCT_NUMBER').agg({'QUANTITY': 'sum', 'UNIT_SALE_PRICE': 'mean'}).reset_index()
cooking_gear_summary = pd.merge(cooking_gear_data, summary, left_on='PRODUCT_NUMBER', right_on='PRODUCT_NUMBER')
sorted_summary = cooking_gear_summary.groupby('PRODUCT_NAME').agg({'QUANTITY': 'sum', 'UNIT_SALE_PRICE': 'mean'}).sort_values(by='QUANTITY', ascending=False)
sorted_summary.reset_index(inplace=True)
sorted_summary

Geef een overzicht met daarin van de verkopers behorend tot de verkoopafdeling uit ‘Amsterdam’ de voornaam en achternaam van de verkoper en zijn laagste en hoogste verkooptarget van 2006. (4 kolommen, 4 rijen) 

Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [ ]:
from datetime import date
date.today().year

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [ ]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 25 jaar in dienst) of een ‘lang in dienst’ (groter gelijk dan 12 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 